In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%cd ..
%pwd

/Users/emeinhardt/Repositories/prague


'/Users/emeinhardt/Repositories/prague'

In [3]:
import prague

In [4]:
from copy import deepcopy
from funcy import *
from functools import reduce
from itertools import combinations, permutations

p0 = lambda t: t[0]
p1 = lambda t: t[1]

In [5]:
# bot = '⊥'
bot = '0'

In [6]:
λ = X0 = set()
A = X1 = set(['a'])
B = X2 = set(['a','b'])
C = X3 = set(['a','b','c'])
D = X4 = set(['a','b','c','d'])
Bool   = set(['+','-'])

In [7]:
X = B
# X = D
Y = Bool

In [8]:
def grand_union(sets, acc=set()):
    return reduce(set.union, sets, acc)

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return tuple(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))

def sort_rel(rel):
    return tuple(sorted(rel, key=p0))

def cartesian_product(L,R):
    return {(x,y) for x in L for y in R}

def relations(L,R):
    return set(powerset(cartesian_product(L,R)))

def rel_to_dict(rel):
    '''
    For finite sets X and Y, transforms 
        rel ⊆ X x Y 
    into
        rel: X → ℙ(Y)
    in the natural way, implemented as a dictionary.
    '''
    dom = set(map(lambda pair: pair[0], rel))
    d = dict()
    for k,v in sorted(rel, key=p0):
        if k in d:
            d[k] = d[k].union({v})
        else:
            d[k] = {v}
    return d

def func_to_dict(functional_rel):
    '''
    For finite sets X and Y and a functional relation 
        rel ⊆ X x Y 
    this returns
        rel: X → Y
    implemented as a dictionary.
    '''
    f = functional_rel
    dR = rel_to_dict(f)
    for k in dR:
        assert len(dR[k]) == 1, f"relation is NOT functional at {k}: {k}→{dR[k]}"
    d = {k:tuple(dR[k])[0] for k in dR}
    return d

def undefined_inputs(func_dic):
    return {k for k,v in func_dic.items() if v == bot}

def domain_of_def(func_dic):
    return set(func_dic.keys()) - undefined_inputs(func_dic)

def is_defined_at(func_dic, x):
    return x in domain_of_def(func_dic)

def is_undefined_at(func_dic, x):
    return x in undefined_inputs(func_dic)

def add_explicit_bot(func_dic, X):
    f = deepcopy(func_dic)
    for x in X:
        if x not in f:
            f[x] = bot
    return f

def same_domain(func_dic_g, func_dic_f):
    g_keys = set(func_dic_g.keys())
    f_keys = set(func_dic_f.keys())
    return g_keys == f_keys

def is_functional(rel):
    d = rel_to_dict(rel)
    multiplicities = walk_values(len, d)
    return max(set(multiplicities.values())) <= 1

def is_total_over(rel, X):
    dom = set(map(lambda pair: pair[0], rel))
    return X.issubset(dom)

def total_functions(L,R):
    rels = relations(L,R)
    total_rels = select(lambda r: is_total_over(r,L), rels)
    fs = select(is_functional, total_rels)
    return walk(sort_rel, fs)

def partial_functions(L,R):
    return walk(sort_rel, total_functions(L, R.union({bot})))

def leq_rel(L,R):
    L_dom, R_dom = set(map(p0, L)), set(map(p0, R))
    assert L_dom == R_dom, f"{L_dom}\nvs.\n{R_dom}"
    return all({yPrime == y or yPrime == bot 
                for (x,y) in R 
                for (xPrime, yPrime) in L 
                if x == xPrime})

def dict_to_rel(func_dic):
    return set(func_dic.items())

def leq_func(f,g):
    assert same_domain(f,g), f"{f}\nvs.\n{g}"
    return leq_rel(dict_to_rel(f), dict_to_rel(g))

In [9]:
relations(X2,X2)

{(),
 (('a', 'a'),),
 (('a', 'a'), ('b', 'a')),
 (('a', 'a'), ('b', 'a'), ('b', 'b')),
 (('a', 'a'), ('b', 'b')),
 (('a', 'b'),),
 (('a', 'b'), ('a', 'a')),
 (('a', 'b'), ('a', 'a'), ('b', 'a')),
 (('a', 'b'), ('a', 'a'), ('b', 'a'), ('b', 'b')),
 (('a', 'b'), ('a', 'a'), ('b', 'b')),
 (('a', 'b'), ('b', 'a')),
 (('a', 'b'), ('b', 'a'), ('b', 'b')),
 (('a', 'b'), ('b', 'b')),
 (('b', 'a'),),
 (('b', 'a'), ('b', 'b')),
 (('b', 'b'),)}

In [10]:
total_functions(X2,X2)

{(('a', 'a'), ('b', 'a')),
 (('a', 'a'), ('b', 'b')),
 (('a', 'b'), ('b', 'a')),
 (('a', 'b'), ('b', 'b'))}

In [11]:
partial_functions(X2,X2)
len(partial_functions(X2,X2))

{(('a', '0'), ('b', '0')),
 (('a', '0'), ('b', 'a')),
 (('a', '0'), ('b', 'b')),
 (('a', 'a'), ('b', '0')),
 (('a', 'a'), ('b', 'a')),
 (('a', 'a'), ('b', 'b')),
 (('a', 'b'), ('b', '0')),
 (('a', 'b'), ('b', 'a')),
 (('a', 'b'), ('b', 'b'))}

9

In [12]:
# FX_rel   = partial_functions(X,X)
# len(FX_rel)
# FX_dict = tuple(map(func_to_dict, FX_rel))

FX_rel   = partial_functions(X,Y)
len(FX_rel)
FX_dict = tuple(map(func_to_dict, FX_rel))

9

In [13]:
FX = FX_dict

In [14]:
FX

({'a': '0', 'b': '-'},
 {'a': '-', 'b': '+'},
 {'a': '-', 'b': '-'},
 {'a': '0', 'b': '+'},
 {'a': '+', 'b': '0'},
 {'a': '+', 'b': '-'},
 {'a': '+', 'b': '+'},
 {'a': '0', 'b': '0'},
 {'a': '-', 'b': '0'})

In [15]:
zero = [f for f in FX if len(domain_of_def(f)) == 0][0]
zero

{'a': '0', 'b': '0'}

In [16]:
my_f = FX[0]
my_g = FX[-1]
my_h = FX[int(len(FX) / 2)]

my_f
my_g
my_h

{'a': '0', 'b': '-'}

{'a': '-', 'b': '0'}

{'a': '+', 'b': '0'}

In [17]:
my_fv = prague.from_feature_dict(my_f, X)
my_gv = prague.from_feature_dict(my_g, X)
my_hv = prague.from_feature_dict(my_h, X)
my_zerov = prague.from_feature_dict(zero, X)
my_fv
my_gv
my_hv
my_zerov

array([ 0, -1], dtype=int8)

array([-1,  0], dtype=int8)

array([1, 0], dtype=int8)

array([0, 0], dtype=int8)

# Functions on functions

In [18]:
def common_keys(f,g):
    return set(f.keys()).intersection(g.keys())

def equalizer(f,g):
    common_dom = common_keys(f,g)
    return {x for x in common_dom if f[x] == g[x]}

def differentiator(f,g):
    common_dom = common_keys(f,g)
    return {x for x in common_dom if f[x] != g[x]}

def common_defined_maps(g,f):
    sameAndNotUndefined = equalizer(g,f) - {bot}
    commonDefinedMaps   = project(f, sameAndNotUndefined)
    return commonDefinedMaps

def unique_defined_maps_left(g,f):
    commonDefinedMaps = common_defined_maps(g,f)
    return {x:g[x] for x in g if (g[x] != bot) and (x not in commonDefinedMaps)}

def unique_defined_maps_right(g,f):
    commonDefinedMaps = common_defined_maps(g,f)
    return {x:f[x] for x in f if (f[x] != bot) and (x not in commonDefinedMaps)}

In [19]:
def mutation_maps(f):
    return {x:f[x] for x in domain_of_def(f) if x != f[x]}
    
def fix_maps(f):
    return {x:f[x] for x in domain_of_def(f) if x == f[x]}

def mutation_set(f):
    return {x for x in domain_of_def(f) if x != f[x]}

def fix_set(f):
    return {x for x in domain_of_def(f) if x == f[x]}

# Priority union and its inverses

## Right priority union

In [20]:
# g+f = f'
def right_priority_union(g,f, treatMissingKeysAsUndefined=False):
    fPrime = deepcopy(f)
    if not treatMissingKeysAsUndefined:
        whereFisUndefined = undefined_inputs(f)
    else:
        allKeys = set(g.keys()).union(set(f.keys()))
        whereFisMissingKeys = {k for k in allKeys if k not in f}
        whereFisUndefined = undefined_inputs(f).union(whereFisMissingKeys)
    for x in whereFisUndefined:
        fPrime[x] = g[x]
    return fPrime

In [21]:
priority_union = right_priority_union

In [22]:
my_f
my_g
'----------'
priority_union(my_f, my_g)
priority_union(my_g, my_f)

{'a': '0', 'b': '-'}

{'a': '-', 'b': '0'}

'----------'

{'a': '-', 'b': '-'}

{'a': '-', 'b': '-'}

In [23]:
sorted(X)[0]
omit(my_f, sorted(X)[0])

'a'

{'b': '-'}

## Left inverse of priority union

In [40]:
# g+f =   f'
#       g\f'

def pru_left_inverse_glb(g,fPrime):
    undefined_inputs_g      = undefined_inputs(g)
    undefined_inputs_fPrime = undefined_inputs(fPrime)
    assert undefined_inputs_fPrime.issubset(undefined_inputs_g)
    
    cdm           = add_explicit_bot(common_defined_maps(g,fPrime), X)
#     print(f"cdm   = {cdm}")
#     gv, fpv       = prague.from_feature_dict(g, X), prague.from_feature_dict(fPrime, X)
#     m             = prague.meet(gv, fpv)
#     print(f"m     = {m}")
    diff          = add_explicit_bot(unique_defined_maps_left(fPrime, cdm), X)
#     print(f"diff = {diff}")
#     kv            = prague.diff(fpv, m)
#     print(f"k     = {kv}")
    
    glb_unordered = add_explicit_bot(diff, X)
    glb = {k:glb_unordered[k] for k in sorted(X)}
    return glb

def pru_left_inverse_lub(g,fPrime):
    cdm  = add_explicit_bot(common_defined_maps(g,fPrime), X)
    diff = add_explicit_bot(unique_defined_maps_left(fPrime, cdm), X)
    
    lub_unordered = add_explicit_bot(priority_union(cdm, diff), X)
    lub = {k:lub_unordered[k] for k in sorted(X)}
    return lub

def pru_left_inverse_interval(g,fPrime):
    undefined_inputs_g      = undefined_inputs(g)
    undefined_inputs_fPrime = undefined_inputs(fPrime)
    if not undefined_inputs_fPrime.issubset(undefined_inputs_g):
        return None

    lub = pru_left_inverse_lub(g,fPrime)
    glb = pru_left_inverse_glb(g,fPrime)
    
    return (lub, glb)

In [25]:
prague.priority_union(my_gv, my_fv)
prague.left_inv(my_gv, prague.priority_union(my_gv, my_fv), True)

array([-1, -1], dtype=int8)

(array([-1, -1], dtype=int8), array([ 0, -1], dtype=int8))

In [26]:
prague.priority_union(my_fv, my_gv)
prague.left_inv(my_fv, prague.priority_union(my_fv, my_gv), True)

array([-1, -1], dtype=int8)

(array([-1, -1], dtype=int8), array([-1,  0], dtype=int8))

In [41]:
print(f"g = {my_g}", f"\nf = {my_f}")
'============================'
my_gPlusf = priority_union(my_g, my_f)
print(f"g + f  =  {my_gPlusf}")
print(f"g \\ f' = {pru_left_inverse_interval(my_g, my_gPlusf)}")
print(f"g \\ f' = {prague.left_inv(my_gv, prague.priority_union(my_gv, my_fv), True)}")
'----------'
my_fPlusg = priority_union(my_f, my_g)
print(f"f + g  =  {my_fPlusg}")
print(f"f \\ g' = {pru_left_inverse_interval(my_f, my_fPlusg)}")
print(f"f \\ g' = {prague.left_inv(my_fv, prague.priority_union(my_fv, my_gv), True)}")

g = {'a': '-', 'b': '0'} 
f = {'a': '0', 'b': '-'}


'============================'

g + f  =  {'a': '-', 'b': '-'}
g \ f' = ({'a': '-', 'b': '-'}, {'a': '0', 'b': '-'})
g \ f' = (array([-1, -1], dtype=int8), array([ 0, -1], dtype=int8))


'----------'

f + g  =  {'a': '-', 'b': '-'}
f \ g' = ({'a': '-', 'b': '-'}, {'a': '-', 'b': '0'})
f \ g' = (array([-1, -1], dtype=int8), array([-1,  0], dtype=int8))


## Right inverse of priority union

In [42]:
# g+f = f'
#       f'/f

def pru_right_inverse_lower_bound(fPrime, f):
    #f must be \leq fPrime
    if not leq_func(f, fPrime):
        return None
    whereFisUndefined = undefined_inputs(f)
    glb_unordered = add_explicit_bot(project(deepcopy(fPrime), 
                                             whereFisUndefined),
                                     X)
    glb = {k:glb_unordered[k] for k in sorted(X)}
    return glb
#     cdm      = add_explicit_bot(common_defined_maps(f,fPrime), X)
#     diff     = add_explicit_bot(unique_defined_maps_left(fPrime, cdm), X)
#     offsetAt = undefined_inputs(fPrime).intersection(undefined_inputs(f))
#     offset   = 

In [29]:
# my_g = {'a': '0', 'b': '0'} 
# my_f = {'a': '-', 'b': '0'}

In [44]:
print(f"g = {my_g}", f"\nf = {my_f}")
'============================'
my_gPlusf = priority_union(my_g, my_f)
print(f"g + f  =  {my_gPlusf}")
print(f"f' / f = ↑{pru_right_inverse_lower_bound(my_gPlusf, my_f)}")
print(f"f' / f = ↑{prague.right_inv(prague.priority_union(my_gv, my_fv), my_fv, True)}")
'----------'
my_fPlusg = priority_union(my_f, my_g)
print(f"f + g  =  {my_fPlusg}")
print(f"g' / f = ↑{pru_right_inverse_lower_bound(my_fPlusg, my_g)}")
print(f"g' / f = ↑{prague.right_inv(prague.priority_union(my_fv, my_gv), my_gv, True)}")

g = {'a': '-', 'b': '0'} 
f = {'a': '0', 'b': '-'}


'============================'

g + f  =  {'a': '-', 'b': '-'}
f' / f = ↑{'a': '-', 'b': '0'}
f' / f = ↑[-1  0]


'----------'

f + g  =  {'a': '-', 'b': '-'}
g' / f = ↑{'a': '0', 'b': '-'}
g' / f = ↑[ 0 -1]


## Partial derivatives wrt priority union

## Partial derivatives wrt left inverse priority union

## Partial derivatives wrt right inverse priority union

# Composition and its inverses

## Composition

In [31]:
#comp

## Left inverse

In [32]:
#left inv

## Right inverse

In [33]:
#right inv

In [34]:
#implementation tests

## Partial derivatives wrt composition

## Partial derivatives wrt left inverse composition

## Partial derivatives wrt right inverse composition

# Property tests

In [35]:
def all_pairs_from(X):
    return {(a,b) for a in X for b in X}

def all_triples_from(X):
    return {(a,b,c) for a in X for b in X for c in X}

def is_commutative_over(op, X, opIsTotal=True):
    f = op
    counterexamples = set()
    for a,b in all_pairs_from(X):
        if opIsTotal:
            if f(a,b) != f(b,a):
                counterexamples.add((a,b, f(a,b), f(b,a)))
        else:
            afb    = f(a,b)
            bfa    = f(b,a)
            if afb is not None and bfa is not None:
                if afb != bfa:
                    counterexamples.add(a,b, afb, bfa)
    return counterexamples

def is_associative_over(op, X, opIsTotal=True):
    f = op
    counterexamples = set()
    for a,b,c in all_triples_from(X):
        if opIsTotal:
            if f(f(a,b),c) != f(a,f(b,c)):
                counterexamples.add((a,b,c, f(a,b), f(f(a,b), c), f(a, f(b,c)), f(b,c)))
        else:
            afb    = f(a,b)
            bfc    = f(b,c)
            afb_fc = None if afb is None else f(afb, c)
            af_bfc = None if bfc is None else f(a  ,bfc)
            if afb_fc is not None and af_bfc is not None:
                if afb_fc != af_bfc:
                    counterexamples.add((a,b,c, afb, afb_fc, af_bfc))
    return counterexamples

def is_universal_right_identity(e, op, X, opIsTotal=True):
    f = op
    counterexamples = set()
    for x in X:
        if opIsTotal:
            if f(x,e) != x:
                counterexamples.add((x, f(x,e)))
        else:
            if f(x,e) != x and f(x,e) is not None:
                counterexamples.add((x, f(x,e)))
    return counterexamples

def is_universal_left_identity(e, op, X, opIsTotal=True):
    f = op
    counterexamples = set()
    for x in X:
        if opIsTotal:
            if f(e,x) != x:
                counterexamples.add((x, f(e,x)))
        else:
            if f(e,x) != x and f(e,x) is not None:
                counterexamples.add((x, f(e,x)))
    return counterexamples


def is_universal_identity(e, op, X, opIsTotal=True):
    f = op
    counterexamples = set()
    for x in X:
        if opIsTotal:
            if f(x,e) != x:
                counterexamples.add((x, e, f(x,e)))
            if f(e,x) != x:
                counterexamples.add((e, x, f(e,x)))
        else:
            if f(x,e) != x and f(x,e) is not None:
                counterexamples.add((x, e, f(x,e)))
            if f(e,x) != x and f(e,x) is not None:
                counterexamples.add((e, x, f(e,x)))
    return counterexamples

# test reproduction axiom
# def 

In [36]:
def lift_total_func_to_hyperop(func_dict):
    f = func_dict
    return {frozenset(k):frozenset(f[k]) for k in f}

## Priority union defines a monoid

In [37]:
#priority union is total over X x Y

In [38]:
#priority union is associative

In [39]:
#the zero function is the universal (two-sided) identity element

## Left inverse of priority union defines a (partial?) hypergroup

## Right inverse of priority union defines a (partial?) hypergroup